In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AdamW
)

from transformers import AutoTokenizer

model_name = "NousResearch/Llama-2-7b-chat-hf"
saved_path = "saved-model/saved_model_20231201-145315"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype
)

re_orig_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

re_orig_model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
re_orig_model.load_state_dict(torch.load(saved_path), strict=False)

In [ ]:
'''predictions = []
original_instructions = []
ground_truth_answer = []

start_marker = '<s>[INST]'
end_marker = '[/INST]'
end_tag = "</s>"

for ri, row in eval_conv.iterrows():
    entire_conv = row["conversations"]
    start_index = entire_conv.find(start_marker)
    end_index = entire_conv.find(end_marker)
    instruction = entire_conv[start_index + len(start_marker):end_index].strip()
    prompt = entire_conv[start_index:end_index + len(end_marker)].strip()
    original_answer = entire_conv[end_index + len(end_marker): len(entire_conv) - len(end_tag) - 1].strip()
    original_instructions.append(instruction)
    ground_truth_answer.append(original_answer)
    print("encoding prompt ...")
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    print("generating response ...")
    outputs = refined_model.generate(input_ids=input_ids, 
        max_new_tokens=156,
        do_sample=True,
    )
    pred = tokenizer.decode(outputs[0])
    predictions.append(pred)
    break

pred_dataframe = pd.DataFrame(zip(original_instructions, ground_truth_answer, predictions), columns=["instructions", "ground truth answers", "generated answers"])
pred_dataframe.to_csv("../data/generated_answers.csv", sep="\t", index=None)

print("Finished generation =======")'''